# 스포티파이 차트 분석
* 평소 스포티파이로 EDM을 들음
* 내가 좋아하는 장르이지만 다른 장르에 비해 인기가 없음
* EDM과 다른 인기곡들 사이에 어떤 차이가 있는지 분석하는 프로젝트

## 1. 분석 목표
### 1. EDM과 그 외 곡들의 핵심 오디오 특성이 있는가?
### 2. 인기 EDM 아티스트의 핵심 오디오 특성이 있는가?

## 2. 데이터 수집
* 스포티파이 차트

### 차트 수집
* 스포티파이 API 활용

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

client_credentials_manager= SpotifyClientCredentials(client_id= '5e31691277084c049eb5fdee39722645',
                                                     client_secret= '32e599441b624e8197b0380595cbb0bb')

sp= spotipy.Spotify(client_credentials_manager= client_credentials_manager)

### 1. 트랙 데이터
* 2023년 트랙 1000곡

In [2]:
artist_name= []
track_name= []
popularity_track= []
artist_id= []
track_id= []

# 1000개 검색
for i in range(0, 1000, 50) :
    # 한 번에 50개씩 검색, 50개 단위로 검색해 저장
    tracks= sp.search(q= 'year:2023', type= 'track', limit= 50, offset= i)

    for i, t in enumerate(tracks['tracks']['items']) :
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity_track.append(t['popularity'])

In [3]:
# 가져온 차트를 데이터프레임으로 변환
import pandas as pd

df_tracks= pd.DataFrame({'artist_name': artist_name,
                         'artist_id': artist_id,
                         'track_name': track_name,
                         'track_id': track_id,
                         'popularity': popularity_track})
df_tracks.head()

,artist_name,artist_id,track_name,track_id,popularity
0,Doja Cat,5cj0lLjcoR7YOSnhnX0Po5,Paint The Town Red,2IGMVunIBsBLtEQyoI1Mu7,100
1,Zach Bryan,40ZNYROS4zLfyyBSs2PGe2,I Remember Everything (feat. Kacey Musgraves),4KULAymBBJcPRpk1yO4dOG,92
2,Gunna,2hlmm7s2ICUX0LVIhVFlZQ,fukumean,4rXLjWdF2ZZpXCVTfWcshS,95
3,Olivia Rodrigo,1McMsnEElThX1knmY4oliG,bad idea right?,3IX0yuEVvDbnqUwMBB3ouC,93
4,Travis Scott,0Y5tJX1MQlPlqiwlOH1tJY,I KNOW ?,6wsqVwoiVH2kde4k4KKAFU,92


### 2. 아티스트 데이터
* 인기, 장르 팔로워 수

In [4]:
popularity_artist= []
genres= []
followers= []

for a_id in df_tracks.artist_id :
    artist= sp.artist(a_id)
    popularity_artist.append(artist['popularity'])
    genres.append(artist['genres'])
    followers.append(artist['followers']['total'])

df_tracks= df_tracks.assign(artist_popularity= popularity_artist,
                 genres= genres,
                 followers= followers)

df_tracks.head()

,artist_name,artist_id,track_name,track_id,popularity,artist_popularity,genres,followers
0,Doja Cat,5cj0lLjcoR7YOSnhnX0Po5,Paint The Town Red,2IGMVunIBsBLtEQyoI1Mu7,100,89,"[dance pop, pop]",26574094
1,Zach Bryan,40ZNYROS4zLfyyBSs2PGe2,I Remember Everything (feat. Kacey Musgraves),4KULAymBBJcPRpk1yO4dOG,92,89,"[classic oklahoma country, modern country pop]",2262988
2,Gunna,2hlmm7s2ICUX0LVIhVFlZQ,fukumean,4rXLjWdF2ZZpXCVTfWcshS,95,85,"[atl hip hop, melodic rap, rap, trap]",5981431
3,Olivia Rodrigo,1McMsnEElThX1knmY4oliG,bad idea right?,3IX0yuEVvDbnqUwMBB3ouC,93,91,[pop],28557065
4,Travis Scott,0Y5tJX1MQlPlqiwlOH1tJY,I KNOW ?,6wsqVwoiVH2kde4k4KKAFU,92,93,"[hip hop, rap, slap house]",24421243


### 3. 오디오 피쳐
* 스포티파이 API 사용
* tempo, danceability, energy, acousticness 등

In [5]:
audio_features= []

for t_id in df_tracks.track_id :
    features= sp.audio_features(t_id)
    audio_features.append(features)

df_features = pd.DataFrame(columns = ['danceability', 'energy', 'loudness',
                                      'speechiness', 'acousticness', 'instrumentalness',
                                      'liveness', 'valence', 'tempo', 'id',
                                      'duration_ms', 'time_signature'])

for item in audio_features :
    for feature in item :
        df_features= df_features.append(feature, ignore_index= True)

df_features.head()

C:\Users\1\AppData\Local\Temp\ipykernel_12736\3069404152.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_features= df_features.append(feature, ignore_index= True)
C:\Users\1\AppData\Local\Temp\ipykernel_12736\3069404152.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_features= df_features.append(feature, ignore_index= True)
C:\Users\1\AppData\Local\Temp\ipykernel_12736\3069404152.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_features= df_features.append(feature, ignore_index= True)
C:\Users\1\AppData\Local\Temp\ipykernel_12736\3069404152.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_features= df_featu

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature,analysis_url,key,mode,track_href,type,uri
0,0.868,0.538,-8.603,0.174,0.269,0.000003,0.0901,0.732,99.968,2IGMVunIBsBLtEQyoI1Mu7,231750,4,https://api.spotify.com/v1/audio-analysis/2IGM...,5.0,1.0,https://api.spotify.com/v1/tracks/2IGMVunIBsBL...,audio_features,spotify:track:2IGMVunIBsBLtEQyoI1Mu7
1,0.429,0.453,-7.746,0.0459,0.554,0.000002,0.102,0.155,77.639,4KULAymBBJcPRpk1yO4dOG,227196,4,https://api.spotify.com/v1/audio-analysis/4KUL...,0.0,1.0,https://api.spotify.com/v1/tracks/4KULAymBBJcP...,audio_features,spotify:track:4KULAymBBJcPRpk1yO4dOG
2,0.847,0.622,-6.747,0.0903,0.119,0,0.285,0.22,130.001,4rXLjWdF2ZZpXCVTfWcshS,125040,4,https://api.spotify.com/v1/audio-analysis/4rXL...,1.0,0.0,https://api.spotify.com/v1/tracks/4rXLjWdF2ZZp...,audio_features,spotify:track:4rXLjWdF2ZZpXCVTfWcshS
3,0.627,0.879,-3.446,0.0955,0.00193,0.000007,0.0647,0.748,129.979,3IX0yuEVvDbnqUwMBB3ouC,184784,4,https://api.spotify.com/v1/audio-analysis/3IX0...,9.0,1.0,https://api.spotify.com/v1/tracks/3IX0yuEVvDbn...,audio_features,spotify:track:3IX0yuEVvDbnqUwMBB3ouC
4,0.927,0.619,-4.441,0.0539,0.0186,0,0.104,0.817,117.995,6wsqVwoiVH2kde4k4KKAFU,211582,4,https://api.spotify.com/v1/audio-analysis/6wsq...,5.0,0.0,https://api.spotify.com/v1/tracks/6wsqVwoiVH2k...,audio_features,spotify:track:6wsqVwoiVH2kde4k4KKAFU


In [6]:
# track table, feature table 병합
df_features.rename(columns= {'id' : 'track_id'}, inplace= True)
df_features.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_id,duration_ms,time_signature,analysis_url,key,mode,track_href,type,uri
0,0.868,0.538,-8.603,0.174,0.269,0.000003,0.0901,0.732,99.968,2IGMVunIBsBLtEQyoI1Mu7,231750,4,https://api.spotify.com/v1/audio-analysis/2IGM...,5.0,1.0,https://api.spotify.com/v1/tracks/2IGMVunIBsBL...,audio_features,spotify:track:2IGMVunIBsBLtEQyoI1Mu7
1,0.429,0.453,-7.746,0.0459,0.554,0.000002,0.102,0.155,77.639,4KULAymBBJcPRpk1yO4dOG,227196,4,https://api.spotify.com/v1/audio-analysis/4KUL...,0.0,1.0,https://api.spotify.com/v1/tracks/4KULAymBBJcP...,audio_features,spotify:track:4KULAymBBJcPRpk1yO4dOG
2,0.847,0.622,-6.747,0.0903,0.119,0,0.285,0.22,130.001,4rXLjWdF2ZZpXCVTfWcshS,125040,4,https://api.spotify.com/v1/audio-analysis/4rXL...,1.0,0.0,https://api.spotify.com/v1/tracks/4rXLjWdF2ZZp...,audio_features,spotify:track:4rXLjWdF2ZZpXCVTfWcshS
3,0.627,0.879,-3.446,0.0955,0.00193,0.000007,0.0647,0.748,129.979,3IX0yuEVvDbnqUwMBB3ouC,184784,4,https://api.spotify.com/v1/audio-analysis/3IX0...,9.0,1.0,https://api.spotify.com/v1/tracks/3IX0yuEVvDbn...,audio_features,spotify:track:3IX0yuEVvDbnqUwMBB3ouC
4,0.927,0.619,-4.441,0.0539,0.0186,0,0.104,0.817,117.995,6wsqVwoiVH2kde4k4KKAFU,211582,4,https://api.spotify.com/v1/audio-analysis/6wsq...,5.0,0.0,https://api.spotify.com/v1/tracks/6wsqVwoiVH2k...,audio_features,spotify:track:6wsqVwoiVH2kde4k4KKAFU


In [7]:
df_tracks= pd.merge(df_tracks, df_features, on= 'track_id', how= 'left')
print(df_tracks.shape)
df_tracks.head()

(1000, 25)


,artist_name,artist_id,track_name,track_id,popularity,artist_popularity,genres,followers,danceability,energy,...,valence,tempo,duration_ms,time_signature,analysis_url,key,mode,track_href,type,uri
0,Doja Cat,5cj0lLjcoR7YOSnhnX0Po5,Paint The Town Red,2IGMVunIBsBLtEQyoI1Mu7,100,89,"[dance pop, pop]",26574094,0.868,0.538,...,0.732,99.968,231750,4,https://api.spotify.com/v1/audio-analysis/2IGM...,5.0,1.0,https://api.spotify.com/v1/tracks/2IGMVunIBsBL...,audio_features,spotify:track:2IGMVunIBsBLtEQyoI1Mu7
1,Zach Bryan,40ZNYROS4zLfyyBSs2PGe2,I Remember Everything (feat. Kacey Musgraves),4KULAymBBJcPRpk1yO4dOG,92,89,"[classic oklahoma country, modern country pop]",2262988,0.429,0.453,...,0.155,77.639,227196,4,https://api.spotify.com/v1/audio-analysis/4KUL...,0.0,1.0,https://api.spotify.com/v1/tracks/4KULAymBBJcP...,audio_features,spotify:track:4KULAymBBJcPRpk1yO4dOG
2,Gunna,2hlmm7s2ICUX0LVIhVFlZQ,fukumean,4rXLjWdF2ZZpXCVTfWcshS,95,85,"[atl hip hop, melodic rap, rap, trap]",5981431,0.847,0.622,...,0.22,130.001,125040,4,https://api.spotify.com/v1/audio-analysis/4rXL...,1.0,0.0,https://api.spotify.com/v1/tracks/4rXLjWdF2ZZp...,audio_features,spotify:track:4rXLjWdF2ZZpXCVTfWcshS
3,Olivia Rodrigo,1McMsnEElThX1knmY4oliG,bad idea right?,3IX0yuEVvDbnqUwMBB3ouC,93,91,[pop],28557065,0.627,0.879,...,0.748,129.979,184784,4,https://api.spotify.com/v1/audio-analysis/3IX0...,9.0,1.0,https://api.spotify.com/v1/tracks/3IX0yuEVvDbn...,audio_features,spotify:track:3IX0yuEVvDbnqUwMBB3ouC
4,Travis Scott,0Y5tJX1MQlPlqiwlOH1tJY,I KNOW ?,6wsqVwoiVH2kde4k4KKAFU,92,93,"[hip hop, rap, slap house]",24421243,0.927,0.619,...,0.817,117.995,211582,4,https://api.spotify.com/v1/audio-analysis/6wsq...,5.0,0.0,https://api.spotify.com/v1/tracks/6wsqVwoiVH2k...,audio_features,spotify:track:6wsqVwoiVH2kde4k4KKAFU


In [8]:
# 불필요한 컬럼 제거
df_tracks.columns

Index(['artist_name', 'artist_id', 'track_name', 'track_id', 'popularity',
       'artist_popularity', 'genres', 'followers', 'danceability', 'energy',
       'loudness', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature',
       'analysis_url', 'key', 'mode', 'track_href', 'type', 'uri'],
      dtype='object')

In [9]:
drop_cols= ['analysis_url', 'key', 'mode', 'track_href', 'type', 'uri']

df_tracks.drop(columns= drop_cols, inplace= True)
print(df_tracks.shape)

(1000, 19)


## 3. 전처리
* 데이터 타입 변경

In [10]:
df_tracks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   artist_name        1000 non-null   object
 1   artist_id          1000 non-null   object
 2   track_name         1000 non-null   object
 3   track_id           1000 non-null   object
 4   popularity         1000 non-null   int64 
 5   artist_popularity  1000 non-null   int64 
 6   genres             1000 non-null   object
 7   followers          1000 non-null   int64 
 8   danceability       1000 non-null   object
 9   energy             1000 non-null   object
 10  loudness           1000 non-null   object
 11  speechiness        1000 non-null   object
 12  acousticness       1000 non-null   object
 13  instrumentalness   1000 non-null   object
 14  liveness           1000 non-null   object
 15  valence            1000 non-null   object
 16  tempo              1000 non-null   object
 

In [11]:
df_tracks.head()

,artist_name,artist_id,track_name,track_id,popularity,artist_popularity,genres,followers,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Doja Cat,5cj0lLjcoR7YOSnhnX0Po5,Paint The Town Red,2IGMVunIBsBLtEQyoI1Mu7,100,89,"[dance pop, pop]",26574094,0.868,0.538,-8.603,0.174,0.269,0.000003,0.0901,0.732,99.968,231750,4
1,Zach Bryan,40ZNYROS4zLfyyBSs2PGe2,I Remember Everything (feat. Kacey Musgraves),4KULAymBBJcPRpk1yO4dOG,92,89,"[classic oklahoma country, modern country pop]",2262988,0.429,0.453,-7.746,0.0459,0.554,0.000002,0.102,0.155,77.639,227196,4
2,Gunna,2hlmm7s2ICUX0LVIhVFlZQ,fukumean,4rXLjWdF2ZZpXCVTfWcshS,95,85,"[atl hip hop, melodic rap, rap, trap]",5981431,0.847,0.622,-6.747,0.0903,0.119,0,0.285,0.22,130.001,125040,4
3,Olivia Rodrigo,1McMsnEElThX1knmY4oliG,bad idea right?,3IX0yuEVvDbnqUwMBB3ouC,93,91,[pop],28557065,0.627,0.879,-3.446,0.0955,0.00193,0.000007,0.0647,0.748,129.979,184784,4
4,Travis Scott,0Y5tJX1MQlPlqiwlOH1tJY,I KNOW ?,6wsqVwoiVH2kde4k4KKAFU,92,93,"[hip hop, rap, slap house]",24421243,0.927,0.619,-4.441,0.0539,0.0186,0,0.104,0.817,117.995,211582,4


In [12]:
df_tracks['artist_name']= df_tracks['artist_name'].astype('string')
df_tracks['artist_id']= df_tracks['artist_id'].astype('string')
df_tracks['track_name']= df_tracks['track_name'].astype('string')
df_tracks['track_id']= df_tracks['track_id'].astype('string')
df_tracks['danceability']= pd.to_numeric(df_tracks['danceability'])
df_tracks['energy']= pd.to_numeric(df_tracks['energy'])
df_tracks['loudness']= pd.to_numeric(df_tracks['loudness'])
df_tracks['speechiness']= pd.to_numeric(df_tracks['speechiness'])
df_tracks['acousticness']= pd.to_numeric(df_tracks['acousticness'])
df_tracks['instrumentalness']= pd.to_numeric(df_tracks['instrumentalness'])
df_tracks['liveness']= pd.to_numeric(df_tracks['liveness'])
df_tracks['valence']= pd.to_numeric(df_tracks['valence'])
df_tracks['tempo']= pd.to_numeric(df_tracks['tempo'])
df_tracks['duration_ms']= pd.to_numeric(df_tracks['duration_ms'])
df_tracks['danceability']= pd.to_numeric(df_tracks['danceability'])
df_tracks['time_signature']= df_tracks['time_signature'].astype('category')

print(df_tracks.info())
df_tracks.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 19 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   artist_name        1000 non-null   string  
 1   artist_id          1000 non-null   string  
 2   track_name         1000 non-null   string  
 3   track_id           1000 non-null   string  
 4   popularity         1000 non-null   int64   
 5   artist_popularity  1000 non-null   int64   
 6   genres             1000 non-null   object  
 7   followers          1000 non-null   int64   
 8   danceability       1000 non-null   float64 
 9   energy             1000 non-null   float64 
 10  loudness           1000 non-null   float64 
 11  speechiness        1000 non-null   float64 
 12  acousticness       1000 non-null   float64 
 13  instrumentalness   1000 non-null   float64 
 14  liveness           1000 non-null   float64 
 15  valence            1000 non-null   float64 
 16  tempo  

,artist_name,artist_id,track_name,track_id,popularity,artist_popularity,genres,followers,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,Doja Cat,5cj0lLjcoR7YOSnhnX0Po5,Paint The Town Red,2IGMVunIBsBLtEQyoI1Mu7,100,89,"[dance pop, pop]",26574094,0.868,0.538,-8.603,0.1740,0.26900,0.000003,0.0901,0.732,99.968,231750,4
1,Zach Bryan,40ZNYROS4zLfyyBSs2PGe2,I Remember Everything (feat. Kacey Musgraves),4KULAymBBJcPRpk1yO4dOG,92,89,"[classic oklahoma country, modern country pop]",2262988,0.429,0.453,-7.746,0.0459,0.55400,0.000002,0.1020,0.155,77.639,227196,4
2,Gunna,2hlmm7s2ICUX0LVIhVFlZQ,fukumean,4rXLjWdF2ZZpXCVTfWcshS,95,85,"[atl hip hop, melodic rap, rap, trap]",5981431,0.847,0.622,-6.747,0.0903,0.11900,0.000000,0.2850,0.220,130.001,125040,4
3,Olivia Rodrigo,1McMsnEElThX1knmY4oliG,bad idea right?,3IX0yuEVvDbnqUwMBB3ouC,93,91,[pop],28557065,0.627,0.879,-3.446,0.0955,0.00193,0.000007,0.0647,0.748,129.979,184784,4
4,Travis Scott,0Y5tJX1MQlPlqiwlOH1tJY,I KNOW ?,6wsqVwoiVH2kde4k4KKAFU,92,93,"[hip hop, rap, slap house]",24421243,0.927,0.619,-4.441,0.0539,0.01860,0.000000,0.1040,0.817,117.995,211582,4


### 인기 차트에 EDM이 있는지 확인

In [13]:
genre_list= pd.Series([x for _list in df_tracks['genres'] for x in _list]).unique().tolist()
# 즐겨듣는 세부 edm 장르가 있는지 확인
print([x for x in genre_list if 'room' in x
                        or 'house' in x
                        or 'edm' in x
                        or 'step' in x])

['slap house', 'bedroom pop', 'bedroom r&b', 'float house', 'big room', 'edm', 'brostep', 'progressive electro house', 'electro house', 'australian house', 'house', 'progressive house', 'tech house', 'escape room', 'dutch edm', 'stutter house', 'edmonton indie']


In [14]:
# big room, edm, progressiv electro house, electro house, house, progressiv house가 포함됨
my_genre= ['big room', 'edm' ,'progressive electro house',
           'electro house', 'house', 'progressive house']
my_genre

['big room',
 'edm',
 'progressive electro house',
 'electro house',
 'house',
 'progressive house']

In [15]:
is_edm= []

for index in df_tracks.index :
    tmp= []
    for i in range(0, len([x for x in df_tracks['genres'][index]])) :
        if [x for x in df_tracks['genres'][index]][i] in my_genre :
            tmp.append('yes')
        else :
            tmp.append('no')
    if 'yes' in tmp :
        is_edm.append('edm')
    else :
        is_edm.append('not_edm')

print(len(is_edm))

df_tracks['is_edm']= is_edm
df_tracks.is_edm.value_counts()

1000


not_edm    988
edm         12
Name: is_edm, dtype: int64

In [16]:
df_tracks.head()

,artist_name,artist_id,track_name,track_id,popularity,artist_popularity,genres,followers,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,is_edm
0,Doja Cat,5cj0lLjcoR7YOSnhnX0Po5,Paint The Town Red,2IGMVunIBsBLtEQyoI1Mu7,100,89,"[dance pop, pop]",26574094,0.868,0.538,-8.603,0.1740,0.26900,0.000003,0.0901,0.732,99.968,231750,4,not_edm
1,Zach Bryan,40ZNYROS4zLfyyBSs2PGe2,I Remember Everything (feat. Kacey Musgraves),4KULAymBBJcPRpk1yO4dOG,92,89,"[classic oklahoma country, modern country pop]",2262988,0.429,0.453,-7.746,0.0459,0.55400,0.000002,0.1020,0.155,77.639,227196,4,not_edm
2,Gunna,2hlmm7s2ICUX0LVIhVFlZQ,fukumean,4rXLjWdF2ZZpXCVTfWcshS,95,85,"[atl hip hop, melodic rap, rap, trap]",5981431,0.847,0.622,-6.747,0.0903,0.11900,0.000000,0.2850,0.220,130.001,125040,4,not_edm
3,Olivia Rodrigo,1McMsnEElThX1knmY4oliG,bad idea right?,3IX0yuEVvDbnqUwMBB3ouC,93,91,[pop],28557065,0.627,0.879,-3.446,0.0955,0.00193,0.000007,0.0647,0.748,129.979,184784,4,not_edm
4,Travis Scott,0Y5tJX1MQlPlqiwlOH1tJY,I KNOW ?,6wsqVwoiVH2kde4k4KKAFU,92,93,"[hip hop, rap, slap house]",24421243,0.927,0.619,-4.441,0.0539,0.01860,0.000000,0.1040,0.817,117.995,211582,4,not_edm


### 오디오 특성
* danceability: 춤추기에 어울리는 정도
* energy: 빠르거나 시끄러운 정도
* loudness: 시끄러운 정도
* speechiness: 대화하는 형식의 정도
* acousticness: 어쿠스틱 장르인 정도
* instrumentalness: 보컬이 없는 곡인 정도
* liveness: 라이브 음원인 정도
* valence: 곡의 가사, 분위기가 긍정적인지의 정도
* tempo: BPM
* duration_ms: 곡의 시간

In [18]:
# 오디오 특성 컬럼
# EDM하면 생각나는 특성들만 추출해서 분석
# danceability, energy, loudness, instrumentalness, valence, tempo
feature_cols= ['danceability', 'energy', 'loudness', 'instrumentalness', 'valence', 'tempo']

df_tracks[feature_cols].describe().T

,count,mean,std,min,25%,50%,75%,max
danceability,1000.0,0.654826,0.152280,0.1460,0.54675,0.664000,0.779000,0.971
energy,1000.0,0.640095,0.169391,0.0436,0.54375,0.657000,0.770250,0.991
loudness,1000.0,-6.249383,2.486683,-19.2880,-7.59625,-5.769000,-4.594500,-0.517
instrumentalness,1000.0,0.014148,0.086527,0.0000,0.00000,0.000001,0.000069,0.938
valence,1000.0,0.490718,0.230909,0.0385,0.30475,0.494500,0.669000,0.971
tempo,1000.0,124.005984,30.997280,56.8290,96.58425,123.943500,146.014000,208.138


In [32]:
# 해당 특성만 포함한 데이터프레임 생성
df_selected= df_tracks[['artist_name', 'artist_id', 'track_name', 'track_id', 'popularity',
                        'artist_popularity', 'genres', 'followers', 'danceability', 'energy',
                        'loudness', 'instrumentalness', 'valence', 'tempo', 'is_edm']]

# 생성한 데이터 프레임을 csv로 저장
df_selected.to_csv('selected_data.csv')